# 🧠 Deepfake Detection with Custom CNN (PyTorch)
This notebook includes:
- Data loading
- CNN model with Dropout
- Training loop
- Validation & Test accuracy
- Saving model as `.pth`


In [2]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [3]:
data_dir = r'C:\Users\admin\Desktop\yapa zeka\deepfake_split_dataset'

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

train_dataset = datasets.ImageFolder(os.path.join(data_dir, 'train'), transform=transform)
val_dataset = datasets.ImageFolder(os.path.join(data_dir, 'val'), transform=transform)
test_dataset = datasets.ImageFolder(os.path.join(data_dir, 'test'), transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [4]:
# 🧠 Define custom CNN model with Dropout
class MyCNN(nn.Module):
    def __init__(self):
        super(MyCNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Dropout(0.25),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
        )
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 16 * 16, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 2)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x

In [8]:
from tqdm import tqdm

# 🚀 Train the model
model = MyCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 30  # 5 epochs
train_losses, val_losses = [], []

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train, total_train = 0, 0
    
    with tqdm(train_loader, desc=f'Training Epoch {epoch+1}/{num_epochs}', unit='batch', ncols=100) as pbar:
        for images, labels in pbar:
            images, labels = images.to(device), labels.to(device) # Verileri grafik kartına aktarın
            optimizer.zero_grad()                           # Önceki gradyanları sıfırla
            outputs = model(images)                         # Form içerisinde veri aktarımı
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()                                # Ağırlıkları güncelle
            running_loss += loss.item()
            
            _, preds = torch.max(outputs, 1)
            correct_train += (preds == labels).sum().item()
            total_train += labels.size(0)
            
            train_accuracy = 100 * correct_train / total_train
            pbar.set_postfix(train_loss=running_loss / (total_train // len(labels)), train_acc=f'{train_accuracy:.4f}%')
    
    train_losses.append(running_loss / len(train_loader))

    model.eval()
    val_loss, correct, total = 0.0, 0, 0

    with tqdm(val_loader, desc=f'Validation Epoch {epoch+1}/{num_epochs}', unit='batch', ncols=100) as pbar:
        with torch.no_grad():
            for images, labels in pbar:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                
                _, preds = torch.max(outputs, 1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)
                
                val_accuracy = 100 * correct / total
                pbar.set_postfix(val_loss=val_loss / (total // len(labels)), val_acc=f'{val_accuracy:.4f}%')
    
    val_losses.append(val_loss / len(val_loader))
    acc = 100 * correct / total
    print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {train_losses[-1]:.4f}, Val Acc: {acc:.2f}%")


Training Epoch 1/30: 100%|█| 4218/4218 [05:20<00:00, 13.15batch/s, train_acc=84.9491%, train_loss=0.
Validation Epoch 1/30: 100%|█| 528/528 [00:49<00:00, 10.56batch/s, val_acc=96.3545%, val_loss=0.0186


Epoch 1/30 - Train Loss: 0.3074, Val Acc: 96.35%


Training Epoch 2/30: 100%|█| 4218/4218 [04:31<00:00, 15.55batch/s, train_acc=95.0628%, train_loss=0.
Validation Epoch 2/30: 100%|█| 528/528 [00:27<00:00, 19.16batch/s, val_acc=97.6052%, val_loss=0.0099


Epoch 2/30 - Train Loss: 0.1184, Val Acc: 97.61%


Training Epoch 3/30: 100%|█| 4218/4218 [04:17<00:00, 16.37batch/s, train_acc=96.2825%, train_loss=0.
Validation Epoch 3/30: 100%|█| 528/528 [00:28<00:00, 18.73batch/s, val_acc=97.6052%, val_loss=0.0103


...

Epoch 28/30 - Train Loss: 0.0416, Val Acc: 98.26%


Training Epoch 29/30: 100%|█| 4218/4218 [04:08<00:00, 16.96batch/s, train_acc=97.9289%, train_loss=0
Validation Epoch 29/30: 100%|█| 528/528 [00:26<00:00, 19.57batch/s, val_acc=98.3640%, val_loss=0.006


Epoch 29/30 - Train Loss: 0.0455, Val Acc: 98.36%


Training Epoch 30/30: 100%|█| 4218/4218 [04:09<00:00, 16.91batch/s, train_acc=97.9764%, train_loss=0
Validation Epoch 30/30: 100%|█| 528/528 [00:26<00:00, 19.65batch/s, val_acc=98.3402%, val_loss=0.005

Epoch 30/30 - Train Loss: 0.0444, Val Acc: 98.34%


In [9]:
# 🧪 Test accuracy
model.eval()
correct, total = 0, 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
print(f"Test Accuracy: {100 * correct / total:.2f}%")

Test Accuracy: 98.09%


In [10]:
torch.save(model.state_dict(), 'deepfake_cnn_model.pth')
print("Model saved as deepfake_cnn_model.pth")

Model saved as deepfake_cnn_model.pth
